In [1]:
import numpy as np
import cv2

In [2]:
def rectify(corner_points):
    corner_points = corner_points.reshape((4, 2))
    new_points = np.zeros((4, 2), dtype=np.float32)
    temp1 = corner_points.sum(1)
    new_points[0] = corner_points[np.argmin(temp1)]
    new_points[2] = corner_points[np.argmax(temp1)]
    temp2 = np.diff(corner_points, axis=1)
    new_points[1] = corner_points[np.argmin(temp2)]
    new_points[3] = corner_points[np.argmax(temp2)]
    return new_points

In [3]:
def get_rectangle(contours):
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    for contour in contours:
        p = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * p, True)

        if len(approx) == 4:
            return approx

In [4]:
def show_images(scanned_image, contour_image, edged_image):
    cv2.imshow("Initial image", contour_image)
    cv2.imshow("Scanned output, colored", scanned_image)
    cv2.imshow("Edged image", edged_image)
    grey_image = cv2.cvtColor(scanned_image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(grey_image, 150, 255, cv2.THRESH_BINARY)
    cv2.imshow("Binary output", binary_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
def get_scanned_image(image, width, height):
    image = cv2.resize(image, (width, height))
    grey_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(grey_image, (5, 5), 0)
    edged_image = cv2.Canny(blurred_image, 0, 50)
    contours, _ = cv2.findContours(edged_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    rectangle = get_rectangle(contours)
    approx_rectangle = rectify(rectangle)
    dimensions = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
    matrix = cv2.getPerspectiveTransform(approx_rectangle, dimensions)
    scanned_image = cv2.warpPerspective(image, matrix, (width, height))
    cv2.drawContours(image, [rectangle], -1, (0, 255, 0), 3)
    return scanned_image, image, edged_image

In [7]:
img = cv2.imread('doc4.jpeg')
scan, c_image, e_image = get_scanned_image(img, 600, 800)
show_images(scan, c_image, e_image)